In [1]:
%autosave 60

Autosaving every 60 seconds


In [2]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [3]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
    pipeline
)
import bitsandbytes

In [4]:
model_id="FacebookAI/roberta-base"
# relace the value with your model: ex <hugging-face-user>/<model-name>

In [5]:
classifier= pipeline('text-classification', model=model_id)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
classifier("HELLO!!")

[{'label': 'LABEL_0', 'score': 0.5355063676834106}]

## **CREATING** **DATASET**

In [8]:
import pandas as pd
df1 = pd.read_csv("/content/sample_data/jira_scrum_role_dataset.csv")
df2 = pd.read_csv("/content/sample_data/role_classification_dataset_long.csv")

In [9]:
dataset = pd.concat([df1,df2], ignore_index=True)

In [10]:
dataset

,description,role
0,Implement responsive UI using React: Implement...,FrontEndEngineer
1,Add new components to the dashboard: Add new c...,FrontEndEngineer
2,Add new components to the dashboard: Add new c...,FrontEndEngineer
3,Fix CSS styling issues for mobile view: Fix CS...,FrontEndEngineer
4,Add new components to the dashboard: Add new c...,FrontEndEngineer
...,...,...
6995,Scale bleeding-edge web-readiness. Develop new...,FrontEndEngineer
6996,Revolutionize scalable solutions. Configure al...,DevOpsEngineer
6997,Syndicate customized paradigms. Improve access...,FrontEndEngineer
6998,Incubate mission-critical architectures. Fix b...,BackendEngineer


In [11]:
dataset_y = dataset['role']
dataset.drop(['role'], inplace=True, axis = 1)

In [12]:
dataset_y

,role
0,FrontEndEngineer
1,FrontEndEngineer
2,FrontEndEngineer
3,FrontEndEngineer
4,FrontEndEngineer
...,...
6995,FrontEndEngineer
6996,DevOpsEngineer
6997,FrontEndEngineer
6998,BackendEngineer


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(dataset, dataset_y, test_size=0.2, random_state=42)

## Preprocessing

In [15]:
cols = ['CloudEngineer', 'DevOpsEngineer', 'AIEngineer',
       'DatabaseDesignEngineer', 'BackendEngineer', 'FrontEndEngineer']

# **TOKENIZING**

In [16]:
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

In [17]:
X_train

,description
1032,Integrate third-party payment gateway: Integra...
6339,Envisioneer front-end e-services. Improve acce...
3886,Optimize cloud resource usage: Optimize cloud ...
2653,Configure Docker containers for services: Conf...
6914,Brand enterprise users. Develop new React comp...
...,...
3772,Migrate services to AWS Lambda: Migrate servic...
5191,Evaluate model performance on validation set: ...
5226,Set up ML pipeline for training and inference:...
5390,Fine-tune GPT model for text generation: Fine-...


In [18]:
train_encodings = tokenizer(list(X_train['description']), padding=True, truncation=True, max_length=256, return_tensors="pt")
test_encodings = tokenizer(list(X_test['description']), padding=True, truncation=True, max_length=256, return_tensors="pt")

In [19]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train_enc = torch.tensor(label_encoder.fit_transform(y_train))
y_test_enc = torch.tensor(label_encoder.transform(y_test))

In [20]:
y_train_enc

tensor([1, 5, 2,  ..., 0, 0, 5])

### Create a Role to Index and Role to Index Mapping

In [21]:
# RoleToIndex = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_).astype(float)))
# IndexToRole = dict(zip(label_encoder.transform(label_encoder.classes_).astype(float), label_encoder.classes_))
IndexToRole = {int(i): role for i, role in zip(label_encoder.transform(label_encoder.classes_), label_encoder.classes_)}
RoleToIndex = {role: int(i) for role, i in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}


In [22]:
RoleToIndex
IndexToRole

{0: 'AIEngineer',
 1: 'BackendEngineer',
 2: 'CloudEngineer',
 3: 'DatabaseDesignEngineer',
 4: 'DevOpsEngineer',
 5: 'FrontEndEngineer'}

Base Model has random labels. The following cell updates the base model with custom labels define above

In [23]:
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": IndexToRole})

In [24]:
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
from torch.utils.data import Dataset,DataLoader
class RoleDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = RoleDataset(train_encodings, y_train_enc)
test_dataset = RoleDataset(test_encodings, y_test_enc)

In [26]:
import time
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

In [27]:
# training_args = TrainingArguments(
#     num_train_epochs=2,
#     per_device_train_batch_size=8,
#     logging_strategy="steps",
#     logging_steps=100,
#     learning_rate=1e-5,
#     weight_decay=0.01,
#     warmup_steps=500,
#     save_strategy="epoch",
#     save_total_limit=2,
#     report_to="tensorboard",
#     push_to_hub=False,
# )

output_dir = f'./summary-training-{str(int(time.time()))}'
training_args = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs=2,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=1000,
    learning_rate=1e-5,
    optim="paged_adamw_8bit",
    logging_strategy="steps",
    logging_steps=100,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=200,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

In [28]:
trainer.train()

Step,Training Loss
100,1.437100
200,0.290800
300,0.078000
400,0.033400
500,0.006400
600,0.028300
700,0.003400
800,0.002800
900,0.002600
1000,0.002500


TrainOutput(global_step=1000, training_loss=0.18853829626739024, metrics={'train_runtime': 349.2007, 'train_samples_per_second': 11.455, 'train_steps_per_second': 2.864, 'total_flos': 152116541856000.0, 'train_loss': 0.18853829626739024, 'epoch': 0.7142857142857143})

In [29]:
model_path = "/content/summary-training-1748936049/checkpoint-1000"

In [31]:
model = RobertaForSequenceClassification.from_pretrained(model_path)

In [32]:
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

In [49]:
input = list(X_test['description'])
X_test

,description
6500,Re-intermediate e-business bandwidth. Fix bugs...
2944,Monitor server metrics with Prometheus: Monito...
2024,Monitor server metrics with Prometheus: Monito...
263,Fix CSS styling issues for mobile view: Fix CS...
4350,Normalize database schema: Normalize database ...
...,...
3484,Set up cloudwatch alarms for EC2 instances: Se...
1860,Implement RESTful APIs with Node.js: Implement...
4974,Add indexes to frequently queried columns: Add...
387,Refactor frontend state management: Refactor f...


In [50]:
text = input[4]

In [51]:
inputs = tokenizer(text, padding=True, truncation=True, max_length=256, return_tensors="pt")

In [52]:
inputs

{'input_ids': tensor([[    0, 45647,  2072,  8503, 47404,    35, 26411,  2072,  8503, 47404,
             4, 28688,  2072, 41614, 22680,    13,   819,     4, 38141,   275,
          3464,    32,  1432,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [53]:
import torch

model.eval()  # Set model to evaluation mode
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()


In [54]:
predicted_role = IndexToRole[predicted_class_id]
print("Predicted role:", predicted_role)

Predicted role: DatabaseDesignEngineer
